# Deliverable 2. Create a Customer Travel Destinations Map.
----
1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Google API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Review the formatting template provided that you'll use to add an information box to each marker in the map. In the pop-up for each city you'll add:

    1. The city name.

    2. The country code.

    3. The weather description and maximum temperature for the city.

15. Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

16. Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

17. Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

18. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.
---

## Ensure that the dependencies and the Google API key is imported correctly.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import Google API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame.

In [2]:
# Set the file path to import the WeatherPy_database.csv file
file_path = "../Weather_Database/WeatherPy_database.csv"

# Load the CSV file into a Pandas DataFrame
city_data_df = pd.read_csv(file_path)

# Display sample data
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather
0,Busselton,AU,-33.6500,115.3333,57.38,85,93,7.36,overcast clouds
1,Victoria,HK,22.2855,114.1577,75.38,63,86,15.99,overcast clouds
2,Vaini,TO,-21.2000,-175.2000,78.96,78,75,8.05,broken clouds
3,Price,US,39.5994,-110.8107,73.11,19,0,0.00,clear sky
4,Hermanus,ZA,-34.4187,19.2345,64.80,92,100,15.14,overcast clouds


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

In [3]:
# Prompt the user to enter the minimum temperature criteria
min_temp = float(input("What is the minimum temperature you would like for your trip? "))

# Prompt the user to enter the maximum temperature criteria
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 80


## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected.

In [4]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc Pandas function
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
                                    
# Display sample data
preferred_cities_df.count()

City          196
Country       192
Lat           196
Lng           196
Max Temp      196
Humidity      196
Cloudiness    196
Wind Speed    196
Weather       196
dtype: int64

## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

In [5]:
# Drop any empty rows in the preferred_cities_df DataFrame and create a new DataFrame.
clean_travel_cities = preferred_cities_df.dropna(axis="index", how="any")

# Display sample data
clean_travel_cities.count()

City          192
Country       192
Lat           192
Lng           192
Max Temp      192
Humidity      192
Cloudiness    192
Wind Speed    192
Weather       192
dtype: int64

## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

In [6]:
# Create DataFrame called hotel_df by copying some columns from the clean_travel_cities DataFrame.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp","Weather", "Lat", "Lng"]].copy()

# Display sample data
hotel_df.head()

,City,Country,Max Temp,Weather,Lat,Lng
1,Victoria,HK,75.38,overcast clouds,22.2855,114.1577
2,Vaini,TO,78.96,broken clouds,-21.2000,-175.2000
3,Price,US,73.11,clear sky,39.5994,-110.8107
5,Luwingu,ZM,73.81,scattered clouds,-10.2621,29.9271
12,Faanui,PF,77.68,light rain,-16.4833,-151.7500


## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

In [7]:
# Add a new empty column, "Hotel Name", to the hotel_df DataFrame
hotel_df["Hotel Name"]= ""


# Display sample data
hotel_df.head()

,City,Country,Max Temp,Weather,Lat,Lng,Hotel Name
1,Victoria,HK,75.38,overcast clouds,22.2855,114.1577,
2,Vaini,TO,78.96,broken clouds,-21.2000,-175.2000,
3,Price,US,73.11,clear sky,39.5994,-110.8107,
5,Luwingu,ZM,73.81,scattered clouds,-10.2621,29.9271,
12,Faanui,PF,77.68,light rain,-16.4833,-151.7500,


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

In [8]:
# Review the parameters to search for a hotel
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key,
}

## Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

In [9]:
# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = hotel_df["Lat"]
    lng = hotel_df["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.") 

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [10]:
# Display sample data
hotel_df.head()

,City,Country,Max Temp,Weather,Lat,Lng,Hotel Name
1,Victoria,HK,75.38,overcast clouds,22.2855,114.1577,
2,Vaini,TO,78.96,broken clouds,-21.2000,-175.2000,
3,Price,US,73.11,clear sky,39.5994,-110.8107,
5,Luwingu,ZM,73.81,scattered clouds,-10.2621,29.9271,
12,Faanui,PF,77.68,light rain,-16.4833,-151.7500,


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

In [11]:
# Drop the rows where there is no Hotel Name.
clean_hotel_df = clean_hotel_df = hotel_df.dropna(axis="index", how="any")


# Display sample data
clean_hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Victoria,HK,75.38,22.2855,114.1577,
2,Vaini,TO,78.96,-21.2000,-175.2000,
12,Faanui,PF,77.68,-16.4833,-151.7500,
14,Vaitape,PF,77.70,-16.5167,-151.7500,
16,Denpasar,ID,76.71,-8.6500,115.2167,
34,Puerto Ayora,EC,75.16,-0.7393,-90.3518,
38,Wanning,CN,75.42,18.8003,110.3967,
40,Kutoarjo,ID,76.26,-7.7169,109.9128,
47,Sehithwa,BW,78.19,-20.4667,22.7167,
51,Santander De Quilichao,CO,77.16,3.0095,-76.4849,


**Note:** If the resulting DataFrame is empty, uncomment the following code to load sample data into the `clean_hotel_df`.

In [12]:
# In case of an empty DataFrame, load the sample data provided
# clean_hotel_df = pd.read_csv("WeatherPy_vacation-backup.csv")
# clean_hotel_df.head(10)

## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

In [13]:
# Set the file name.
output_data_file = # YOUR CODE HERE

# Create a CSV file by using the clean_hotel_df DataFrame
# YOUR CODE HERE

SyntaxError: invalid syntax (900737826.py, line 2)

### Review the formatting template provided that you'll use to add an information box to each marker in the map.

In the pop-up for each city you'll add: 

1. The city name

2. The country code

3. The weather description and maximum temperature for the city

In [ ]:
# Review the formatting template provided
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

## Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

In [ ]:
# Get the data from each row in the clean_hotel_df DataFrame, add it to the formatting template, and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# Display sample data
hotel_info[:10]

## Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

In [ ]:
# Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

# Display sample data
locations.head(10)

## Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

In [ ]:
# Add a marker layer for each city to the map. 
# YOUR CODE HERE

# Create a figure to add the Google map as a layer
# YOUR CODE HERE

# Display the figure containing the map
# YOUR CODE HERE